In [ ]:
!pip install gymnasium
!pip install swig
!pip install gymnasium[classic-control]
!pip install moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00


## Pendulum Q-learning agent

In [ ]:
from collections import defaultdict
import gymnasium as gym
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

class PendulumAgent:
    def __init__(
        self,
        env: gym.Env,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
        n_bins: int = 10,
        n_action_bins: int = 10,
    ):
        self.env = env
        self.q_values = defaultdict(lambda: np.zeros(n_action_bins))
        self.lr = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.n_bins = n_bins
        self.n_action_bins = n_action_bins
        self.bins = self.create_bins()
        self.action_bins = np.linspace(env.action_space.low[0], env.action_space.high[0], n_action_bins)
        self.training_error = []

    def create_bins(self):
        bin_limits = [
            [-1.0, 1.0],  # cos(theta)
            [-1.0, 1.0],  # sin(theta)
            [-8.0, 8.0],  # angular velocity
        ]
        bins = [np.linspace(limits[0], limits[1], self.n_bins - 1) for limits in bin_limits]
        return bins

    def discretize(self, obs):
        return tuple(np.digitize(obs[i], self.bins[i]) for i in range(len(obs)))

    def get_discrete_action(self, action):
        return np.digitize(action, self.action_bins) - 1

    def get_continuous_action(self, discrete_action):
        return self.action_bins[discrete_action]

    def get_action(self, obs: np.ndarray) -> float:
        discrete_state = self.discretize(obs)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()  # explore (returns continuous action)
        else:
            discrete_action = int(np.argmax(self.q_values[discrete_state]))  # exploit
            return self.get_continuous_action(discrete_action)

    def update(self, obs: np.ndarray, action: float, reward: float, terminated: bool, next_obs: np.ndarray):
        discrete_state = self.discretize(obs)

        discrete_action = self.get_discrete_action(action)
        discrete_next_states = self.discretize(next_obs)
        future_q_value = (not terminated) * np.max(self.q_values[discrete_next_states])
        temporal_difference = (
            reward + self.discount_factor * future_q_value - self.q_values[discrete_state][discrete_action]
        )
        self.q_values[discrete_state][discrete_action] += self.lr * temporal_difference
        self.training_error.append(temporal_difference)


    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)


import moviepy.editor as mpy
import time
import os

env_name = "Pendulum-v1"
if not os.path.exists(f"videos_{env_name}"):
  os.mkdir(f"videos_{env_name}")

def record_video(frames, episode_num, env_name):
    """Saves a video from a list of frames."""
    clip = mpy.ImageSequenceClip(frames, fps=30)
    clip.write_videofile(f"videos_{env_name}/episode_{episode_num}.mp4", codec="libx264")


# Parameters
learning_rate = 0.05
n_episodes = 250000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)
final_epsilon = 0.01
video_interval = 5000  # Record every 1000 episodes
nr_bins = 50

# Create environment
env = gym.make("Pendulum-v1", render_mode="rgb_array", g=9.81)
agent = PendulumAgent(
    env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
    n_bins=nr_bins,
    n_action_bins=nr_bins,
)

episode_rewards = []

# Training loop
for episode in tqdm(range(n_episodes + 1)):
    obs, info = env.reset()
    done = False
    total_reward = 0
    frames = []  # To store frames for video

    while not done:
        action = agent.get_action(obs)
        next_obs, reward, terminated, truncated, info = env.step([action])
        next_obs = np.squeeze(next_obs)

        # Record the frame if it's a video episode
        if episode % video_interval == 0:
            frames.append(env.render())

        agent.update(obs, action, reward, terminated, next_obs)
        total_reward += reward
        done = terminated or truncated
        obs = next_obs

    episode_rewards.append(total_reward)
    agent.decay_epsilon()

    # Save the video for this episode if needed
    if episode % video_interval == 0:
        record_video(frames, episode, env_name)

env.close()

episode_rewards_flat = []
for episode_reward in episode_rewards:
  if isinstance(episode_reward, float):
    episode_rewards_flat.append(episode_reward)
  else:
    episode_rewards_flat.append(episode_reward[0])

# Plot the episode rewards
plt.figure(figsize=(12, 6))
plt.plot(episode_rewards_flat, label="Episode Reward")
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("Training Progress: Episode Rewards Over Time (Pendulum-v1)")
plt.legend()
plt.grid()
plt.show()



  0%|          | 0/250001 [00:01<?, ?it/s]

Moviepy - Building video videos_Pendulum-v1/episode_0.mp4.
Moviepy - Writing video videos_Pendulum-v1/episode_0.mp4




  0%|          | 3/250001 [00:02<44:05:11,  1.58it/s] 

Moviepy - Done !
Moviepy - video ready videos_Pendulum-v1/episode_0.mp4


  0%|          | 29/250001 [00:04<5:27:57, 12.70it/s]WARNING:py.warnings:<ipython-input-14-5060b8070e1d>:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.q_values[discrete_state][discrete_action] += self.lr * temporal_difference

  2%|▏         | 4999/250001 [04:47<5:29:55, 12.38it/s]

...

 80%|███████▉  | 199997/250001 [2:33:17<27:46, 30.01it/s]

Moviepy - Building video videos_Pendulum-v1/episode_200000.mp4.
Moviepy - Writing video videos_Pendulum-v1/episode_200000.mp4




 80%|████████  | 200004/250001 [2:33:19<2:47:46,  4.97it/s]

Moviepy - Done !
Moviepy - video ready videos_Pendulum-v1/episode_200000.mp4


 81%|████████  | 201257/250001 [2:34:08<33:34, 24.20it/s]

In [ ]:
np.save(f"learning_episode_rewards_50_250000.npy", episode_rewards_flat)

In [ ]:
def evaluate_pendulum(agent, env, num_episodes=100):
    total_rewards = []
    for _ in range(num_episodes):
        obs, info = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = agent.get_action(obs)
            obs, reward, terminated, truncated, info = env.step([action])
            total_reward += reward
            done = terminated or truncated
            obs = np.squeeze(obs)
        if type(total_reward) != np.ndarray:
          total_rewards.append(total_reward)
        else:
          total_rewards.append(np.squeeze(total_reward[0]))
    return np.mean(total_rewards)

average_reward_eval = evaluate_pendulum(agent, env, num_episodes=1000)

print(f"Average reward over 1000 episodes: {average_reward_eval}")

## Acrobot Q-learning agent

In [ ]:
from collections import defaultdict
import gymnasium as gym
import numpy as np
from tqdm import tqdm
import time
import os
import matplotlib.pyplot as plt
import moviepy.editor as mpy

env_name = "Acrobot-v1"
if not os.path.exists(f"videos_{env_name}"):
    os.mkdir(f"videos_{env_name}")

def record_video(frames, episode_num, env_name):
    """Saves a video from a list of frames."""
    clip = mpy.ImageSequenceClip(frames, fps=60)
    clip.write_videofile(f"videos_{env_name}/episode_{episode_num}.mp4", codec="libx264")


class AcrobotAgent:
    def __init__(
        self,
        env: gym.Env,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
        n_bins: int = 10,
    ):
        self.env = env
        self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))
        self.lr = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.n_bins = n_bins
        self.bins = self.create_bins()
        self.training_error = []

    def create_bins(self):
        # Observation space: [cos(theta1), sin(theta1), cos(theta2), sin(theta2), theta1_dot, theta2_dot]
        bin_limits = [
            [-1.0, 1.0],  # cos(theta1)
            [-1.0, 1.0],  # sin(theta1)
            [-1.0, 1.0],  # cos(theta2)
            [-1.0, 1.0],  # sin(theta2)
            [-12.0, 12.0],  # theta1_dot
            [-36.0, 36.0],  # theta2_dot
        ]
        bins = [np.linspace(limits[0], limits[1], self.n_bins - 1) for limits in bin_limits]
        return bins

    def discretize(self, obs):
        return tuple(np.digitize(obs[i], self.bins[i]) for i in range(len(obs)))

    def get_action(self, obs: np.ndarray) -> int:
        discrete_state = self.discretize(obs)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()  # Explore
        else:
            return int(np.argmax(self.q_values[discrete_state]))  # Exploit

    def update(self, obs: np.ndarray, action: int, reward: float, terminated: bool, next_obs: np.ndarray):
        discrete_state = self.discretize(obs)
        discrete_next_state = self.discretize(next_obs)
        future_q_value = (not terminated) * np.max(self.q_values[discrete_next_state])
        temporal_difference = (
            reward + self.discount_factor * future_q_value - self.q_values[discrete_state][action]
        )
        self.q_values[discrete_state][action] += self.lr * temporal_difference
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)


# Training loop
# Hyperparameters
learning_rate = 0.05
n_episodes = 13000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)
final_epsilon = 0.01
video_interval = 500  # Save video every 100 episodes

# Create environment
env = gym.make("Acrobot-v1", render_mode="rgb_array")
agent = AcrobotAgent(
    env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)

episode_rewards = []

for episode in tqdm(range(n_episodes + 1)):
    obs, info = env.reset()
    done = False
    total_reward = 0
    frames = []

    while not done:
        frame = env.render()
        if episode % video_interval == 0:
            frames.append(frame)

        action = agent.get_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        reward = reward if not terminated else -100  # Penalize failure
        agent.update(obs, action, reward, terminated, next_obs)

        total_reward += reward
        done = terminated or truncated
        obs = next_obs

    episode_rewards.append(total_reward)
    agent.decay_epsilon()

    if episode % video_interval == 0:
        record_video(frames, episode, env_name)

env.close()

# Plot the episode rewards
plt.figure(figsize=(12, 6))
plt.plot(episode_rewards, label="Episode Reward")
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("Training Progress: Episode Rewards Over Time (Acrobot-v1)")
plt.legend()
plt.grid()
plt.show()


  0%|          | 0/13001 [00:02<?, ?it/s]

Moviepy - Building video videos_Acrobot-v1/episode_0.mp4.
Moviepy - Writing video videos_Acrobot-v1/episode_0.mp4




  0%|          | 1/13001 [00:05<20:23:00,  5.64s/it]

Moviepy - Done !
Moviepy - video ready videos_Acrobot-v1/episode_0.mp4


  0%|          | 26/13001 [00:53<7:21:18,  2.04s/it]


KeyboardInterrupt: 

In [ ]:
#!zip -r /content/videos_CartPole-v1/cartpole_combined_videos.zip /content/videos_CartPole-v1/*
#!zip -r /content/videos_Pendulum-v1/pendulum_combined_videos.zip /content/videos_Pendulum-v1/*
#!zip -r /content/videos_Acrobot-v1/acrobot_combined_videos.zip /content/videos_Acrobot-v1/*

  adding: content/videos_CartPole-v1/episode_0.mp4 (deflated 20%)
  adding: content/videos_CartPole-v1/episode_1000.mp4 (deflated 22%)
  adding: content/videos_CartPole-v1/episode_1500.mp4 (deflated 21%)
  adding: content/videos_CartPole-v1/episode_2000.mp4 (deflated 20%)
  adding: content/videos_CartPole-v1/episode_2500.mp4 (deflated 19%)
  adding: content/videos_CartPole-v1/episode_3000.mp4 (deflated 20%)
  adding: content/videos_CartPole-v1/episode_3500.mp4 (deflated 24%)
  adding: content/videos_CartPole-v1/episode_4000.mp4 (deflated 20%)
  adding: content/videos_CartPole-v1/episode_4500.mp4 (deflated 21%)
  adding: content/videos_CartPole-v1/episode_5000.mp4 (deflated 21%)
  adding: content/videos_CartPole-v1/episode_500.mp4 (deflated 21%)
  adding: content/videos_CartPole-v1/episode_5500.mp4 (deflated 21%)
  adding: content/videos_CartPole-v1/episode_6000.mp4 (deflated 18%)
  adding: content/videos_CartPole-v1/episode_6500.mp4 (deflated 23%)
  adding: content/videos_CartPole-v1/e